In [1]:
import os
import cv2
from tqdm import tqdm

In [2]:
dataset_path = "./datasets/beetle-tracking/Detection_data/"

In [3]:
##Get all .txt files
def _get_all_txt(folder_path):
    f = []
    idx = 0
    if os.path.exists(folder_path):
        for x in os.listdir(folder_path):
            name, ext = os.path.splitext(x)
            if ext == '.txt':
                #print(x)
                f.append(os.path.join(folder_path,x))
    else:
        print("%s does not exist!"%(folder_path))
    
    return f
    
txt_files = _get_all_txt(dataset_path)

In [4]:
##Read description file and transfer the content to py dict.
def _read_box_desc(file_name):
    f = open(file_name,'r')
    result = dict()
    res = dict()
    cnt = 0
    result['fn'] = file_name
    vname, _ = os.path.splitext(file_name)
    result['name'] = os.path.split(vname)[-1]
    result['name'] = result['name'].replace(" ", "")
    result['vfn'] = "%s.avi"%vname[:-6]
    
    line = f.readline()
    while line:
        cnt = cnt+1
        line = eval("[%s]"%line)
        if len(line)>2:
            print("err", line)
            return result
        
        res[line[0]] = line[1]
        line = f.readline()
    
    f.close()
    result['res'] = res
    result['cnt'] = cnt
    return result

r = _read_box_desc(txt_files[0])
print(r['fn'])
print(r['vfn'])
print(r['cnt'])
print(r['name'])
#print(r['res'])

./datasets/beetle-tracking/Detection_data/[CH03] 2017-08-12 21.15.00_x264_label.txt
./datasets/beetle-tracking/Detection_data/[CH03] 2017-08-12 21.15.00_x264.avi
300
[CH03]2017-08-1221.15.00_x264_label


In [26]:
## for debugging
def _pick_color(id, bCorrect):
    if bCorrect==False:
        cr = (0,0,255)
    elif id==1:
        cr = (0,255,0)
    elif id==2: 
        cr = (255,0,0)
    elif id==3: 
        cr = (0,255,255)
    elif id==4: 
        cr = (255,255, 0)
    else:
        cr = (255,255,255)
    
    return cr

In [22]:
#from pascal_voc_writer import Writer

In [23]:
def _build_filename(vname, cnt, idx):
    ostr = "%s_f%05d_%05d"%(vname, cnt, idx)
    return ostr

In [8]:
def _correct_box(s, e):
    if s <0:s = 0
    if e <0:e = 0
    if s>=e:
        print("incorrect box val %d:%d"%(s,e))
        s = e = 0
    return s,e
        
        

In [9]:
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import numpy as np

BNET=load_model('./Classification/model-resnet50-0321-2.h5')

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [27]:
def predict_Beetle_id(net, img):
    img = cv2.resize(img, (200,200), interpolation=cv2.INTER_CUBIC)
    img = np.expand_dims(img, axis = 0)
    pred = net.predict(img)[0]
    pred_y = np.argmax(pred)
    return pred_y+1

In [30]:
## Read target video files and extract frames as image file according to the description file
def _classify_image(img_prefix, desc, out_txt_name, out_folder, wBox=True):
    ##{out_folder}/images
    ##{out_folder}/annotations
    ##{out_folder}/logs/{lname}
    
    print(desc['vfn'], desc['name'])
    
    #input video path
    vf = desc['vfn']
    #output training_set name
    lname = desc['name']
    
    if not os.path.isfile(vf):
        print("wrong video file")
        return
    
    #output log folder path
    out_log_folder = os.path.join(out_folder,'log', lname)
    if not os.path.isdir(out_log_folder):
        print("create out log  dir %s"%out_log_folder)
        os.makedirs(out_log_folder)
    else:
        print("%s already existed, pass this avi"%lname)
        return

    #output image folder
    out_img_folder = os.path.join(out_folder,'images')
    if not os.path.isdir(out_img_folder):
        #print("create out log  dir %s"%out_log_folder)
        os.makedirs(out_img_folder)

    #output anno folder
    #out_anno_folder = os.path.join(out_folder,'annotations')
    #if not os.path.isdir(out_anno_folder):
    #    os.makedirs(out_anno_folder)        
        
    #output training set index file
    if out_txt_name == '':
        out_txt_name = 'class.txt'
        out_txt_log_name = 'class_log.txt'
    else:
        out_txt_name = '%s.txt'%out_txt_name
        out_txt_log_name = '%s_log.txt'%out_txt_name
        
    out_txt_path = os.path.join(out_log_folder, out_txt_name)
    out_txt_log_path = os.path.join(out_log_folder, out_txt_log_name)
    
    #print("output img\n%s \ntxt\n%s \nlog\n%s"%(out_img_folder,out_txt_path,out_txt_log_path))
    
    fw = open(out_txt_path,'w')
    fw_log = open(out_txt_log_path,'w')
    
    #read video files
    cap = cv2.VideoCapture(vf)
    timeout = 0
    while not cap.isOpened():
        #cap = cv2.VideoCapture(vf)
        cv2.waitKey(1000)
        print("Wait for the header")
        timeout = timeout +1
        if(timeout>10):
            break
    #print(cap.isOpened())
    total_frames = cap.get(7) ## CV_CAP_PROP_FRAME_COUNT
    print("total_frames %d"%total_frames)    
    
    res = desc['res']
    fcnt = 0
    #video_out = os.path.join(out_img_folder, "Out_%s_box.avi"%lname)
    #video_writer = cv2.VideoWriter(video_out,
    #                       cv2.VideoWriter_fourcc(*'MPEG'), 
    #                       15.0, 
    #                       (1280, 760))
    #print(video_out)
    #print(video_writer)
    #return
    cnt_correct = 0
    cnt_wrong = 0
    for f_idx in tqdm(res.keys()):#go through specific frames and save it
        if f_idx>total_frames:
            print("parse error. incorrect frame idx %d-%d"%(f_idx,total_frames))
            return
        
        #print(f_idx, res[f_idx])
        fw_log.write("%s %s\n"%(_build_filename("",fcnt, f_idx),str(res[f_idx])))
        cap.set(1, f_idx) #CV_CAP_PROP_POS_FRAMES, frame_num
        ret, frame = cap.read()
        if ret == False:
            print("%s read failed, ignore it"%vf)
            continue
            
        fcnt = fcnt +1
        img_path = os.path.join(out_img_folder, "%s.jpg"%(_build_filename(lname,fcnt,f_idx)))
        cv2.imwrite(img_path, frame)
        out_str = "%s"%(os.path.abspath(img_path))
        frame_box = frame.copy()
        TYPE_BUG = 0
        TYPE_UNKNOWN = 1
        #writer = Writer(img_path, 1280, 720)

        for bug_id,left,right in res[f_idx]:
            #print(bug_id,left,right)
            y0 , y1 = _correct_box(left[1], right[1])
            x0 , x1 = _correct_box(left[0], right[0])
            h = y1 - y0
            w = x1 - x0
            if h > w:
                w = h
            else:
                h = w
            frame_cropped_box = frame[y0:y0+h, x0:x0+w]
            pid = predict_Beetle_id(BNET, frame_cropped_box)   
            bCorrect = False
            if bug_id!=5 and pid != bug_id :
                #print("wrong classification %d (%d)"%(pid, bug_id))
                fw_log.write("wrong classification %d (%d)\n"%(pid, bug_id))
                cnt_wrong = cnt_wrong + 1
            else:
                cnt_correct = cnt_correct + 1
                bCorrect = True

            img_path_cropped = os.path.join(out_img_folder, "%s_box_%d.jpg"%(_build_filename(lname,fcnt,f_idx), bug_id))            
            cv2.imwrite(img_path_cropped, frame_cropped_box)            
                
            cr = _pick_color(bug_id, bCorrect)
            cv2.rectangle(frame_box,left,right,cr,2) 
            if not bCorrect:
                txt = "%d(%d)"%(pid, bug_id)
            else:
                txt = str(bug_id)
            cv2.putText(frame_box, txt, left, 0,1,cr)
            out_str = "%s %d,%d,%d,%d,%d"%(out_str, left[0], left[1], right[0], right[1], TYPE_BUG)
            
            #writer.addObject('beetle', left[0], left[1], right[0], right[1])
        
        
        #anno_path = os.path.join(out_anno_folder, "%s.xml"%(_build_filename(lname,fcnt,f_idx)))
        #writer.save(anno_path)
        #out_str = "%s\n%s\n"%(out_str,anno_path)
        out_str = "%s\n"%(out_str)
        fw.write(out_str)
        
        if wBox:
            img_path_b = os.path.join(out_img_folder, "%s_box.jpg"%(_build_filename(lname,fcnt,f_idx)))
            cv2.imwrite(img_path_b, frame_box)
            #video_writer.write(frame_box)
    print("accur: %02f"%(cnt_correct*100/(cnt_wrong+cnt_correct)))
    fw.close()
    fw_log.close()
    cap.release()    
    #video_writer.release()
    cv2.destroyAllWindows()
    print("capture done %s"%vf)
    return 

#ff = _capture_image_from_desc(r, '', './ds_train/')

In [32]:
##main
txt_files = _get_all_txt(dataset_path)
idx = 0
for txt in sorted(txt_files):
    idx = idx+1
    d = _read_box_desc(txt)
    _classify_image(idx, d, '', './ds_train_cropped/',True)
    if idx>7:
        break

  0%|          | 0/301 [00:00<?, ?it/s]

./datasets/beetle-tracking/Detection_data/[CH01] 2016-10-14 19.20.00_x264.avi [CH01]2016-10-1419.20.00_x264_label
[CH01]2016-10-1419.20.00_x264_label already existed, pass this avi
./datasets/beetle-tracking/Detection_data/[CH01] 2016-10-20 19.50.00_x264.avi [CH01]2016-10-2019.50.00_x264_label
[CH01]2016-10-2019.50.00_x264_label already existed, pass this avi
./datasets/beetle-tracking/Detection_data/[CH01] 2016-10-28 19.20.00_x264.avi [CH01]2016-10-2819.20.00_x264_label
[CH01]2016-10-2819.20.00_x264_label already existed, pass this avi
./datasets/beetle-tracking/Detection_data/[CH01] 2016-11-05 20.20.00_x264.avi [CH01]2016-11-0520.20.00_x264_label
[CH01]2016-11-0520.20.00_x264_label already existed, pass this avi
./datasets/beetle-tracking/Detection_data/[CH01] 2016-11-11 20.40.00_x264.avi [CH01]2016-11-1120.40.00_x264_label
[CH01]2016-11-1120.40.00_x264_label already existed, pass this avi
./datasets/beetle-tracking/Detection_data/[CH01] 2017-01-20 20.15.00_x264.avi [CH01]2017-01-202

100%|██████████| 301/301 [02:26<00:00,  2.24it/s]

accur: 69.063387
capture done ./datasets/beetle-tracking/Detection_data/[CH01] 2017-08-12 19.15.00_x264.avi
